In [39]:
import pandas as pd

Data = pd.DataFrame()

for re in result_101:
    office_name = re['officeName']
    title = re['title']
    summary = re['summary']
    office_id = re['officeId'] # 신문사에 대한 코드
    article_id = re['articleId'] # 기사 제목에 대한 코드
    
    base_url = 'https://n.news.naver.com/mnews/article/'
    article_url = base_url + office_id + '/' + article_id
    
    new_df = pd.DataFrame({'신문사': office_name, '제목': title, '기사 요약': summary,
                          'URL': article_url}, index=[0])
    
    Data = pd.concat([Data, new_df], ignore_index=True)


In [62]:
import requests
import json
import pandas as pd
import time
import random

Category_code = ['103', '101', '102',  '105', '104', '100'] # 경제, 사회, IT/과학, 세계
Category = ['생활/문화', '경제', '사회', 'IT/과학', '세계', '정치']

Data = pd.DataFrame()

for cate_code, category in zip(Category_code, Category):
    for page in range(1, 11):
        url = f'https://news.naver.com/main/mainNews.naver?sid1={cate_code}&date=%2000:00:00&page={page}'
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        response = requests.get(url, headers = headers) # 페이지 불러오기
        ticket = response.json()

        airsResult = ticket['airsResult'] # 1차 진입
        dic_result = json.loads(airsResult) # 딕셔너리 전환

        result = dic_result['result']
        result_with_code = result[cate_code]

        base_url = 'https://n.news.naver.com/mnews/article/'

        for res in result_with_code:
            title = res['title']
            summary = res['summary']
            office_id = res['officeId']
            article_id = res['articleId']
            office_name = res['officeName']
            article_url = base_url + office_id + '/' + article_id

            new_df = pd.DataFrame({'카테고리': category, '제목': title, 
                                  '언론사': office_name, 'URL': article_url}, index=[0])
            
            Data = pd.concat([Data, new_df], ignore_index = True)
            
        print(f'{page} of {category} Page was Done')
        time.sleep(random.uniform(1, 2)) # 한 페이지가 끝난 후 1초에서 2초 사이 휴식     


1 of 생활/문화 Page was Done
2 of 생활/문화 Page was Done
3 of 생활/문화 Page was Done
4 of 생활/문화 Page was Done
5 of 생활/문화 Page was Done
6 of 생활/문화 Page was Done
7 of 생활/문화 Page was Done
8 of 생활/문화 Page was Done
9 of 생활/문화 Page was Done
10 of 생활/문화 Page was Done
1 of 경제 Page was Done
2 of 경제 Page was Done
3 of 경제 Page was Done
4 of 경제 Page was Done
5 of 경제 Page was Done
6 of 경제 Page was Done
7 of 경제 Page was Done
8 of 경제 Page was Done
9 of 경제 Page was Done
10 of 경제 Page was Done
1 of 사회 Page was Done
2 of 사회 Page was Done
3 of 사회 Page was Done
4 of 사회 Page was Done
5 of 사회 Page was Done
6 of 사회 Page was Done
7 of 사회 Page was Done
8 of 사회 Page was Done
9 of 사회 Page was Done
10 of 사회 Page was Done
1 of IT/과학 Page was Done
2 of IT/과학 Page was Done
3 of IT/과학 Page was Done
4 of IT/과학 Page was Done
5 of IT/과학 Page was Done
6 of IT/과학 Page was Done
7 of IT/과학 Page was Done
8 of IT/과학 Page was Done
9 of IT/과학 Page was Done
10 of IT/과학 Page was Done
1 of 세계 Page was Done
2 of 세계 Page was Done
3 of 세계 Page

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


Contents_data= pd.DataFrame()

for category, journal, url in zip(Data['카테고리'], Data['언론사'], Data['URL']):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    
    response = requests.get(url, headers = headers) # 페이지 불러오기
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find('h2', attrs={'class' : 'media_end_head_headline'}).get_text() #  제목
    contents = soup.find('div', attrs={'id' : 'dic_area'}).get_text() # 기사 내용
    
    new_contents_data = pd.DataFrame({'Category': category, 'Title': title,  'Contents': contents,
                                     }, index=[0])
    
    Contents_data = pd.concat([Contents_data, new_contents_data])
    time.sleep(random.uniform(1, 2)) # 한 페이지가 끝난 후 1초에서 2초 사이 휴식
    
Contents_data.head()

In [66]:
Contents_data.to_csv("news.csv", mode='w')